In [2]:
# general tools
import os
import re
import sys
import netCDF4 as nc
from glob import glob

import numpy as np
from datetime import datetime, timedelta

In [3]:
lead = 2

In [9]:
model_tag = 'alt' #args['model_tag']
filepath_vec = "/glade/work/ksha/NCAR/"

In [5]:
if (lead < 9) or (lead > 18):
    path_name = '/glade/scratch/ksha/DATA/NCAR_batch_v3/'
else:
    path_name = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v3/'

In [6]:
filename_valid = sorted(glob("{}VALID*lead{}.npy".format(path_name, lead)))

In [10]:
#filename_valid
valid_lead = np.load('{}TEST_pp15_pred_lead{}_{}.npy'.format(filepath_vec, lead, model_tag), allow_pickle=True)[()]

In [11]:
VALID_lead_y = valid_lead['y_true']

In [14]:
IND_VALID_lead = np.load('/glade/work/ksha/NCAR/IND_VALID_lead_full.npy', allow_pickle=True)[()]
VALID_ind = IND_VALID_lead['lead{}'.format(lead)]

In [18]:
L = len(VALID_ind)

filename_valid_pick = []

VALID_Y = np.zeros(L)

for i in range(L):
    
    ind_lead = int(VALID_ind[i])
    filename_valid_pick.append(filename_valid[ind_lead])

    if 'pos' in filename_valid[ind_lead]:
        if VALID_lead_y[ind_lead] == 1.0:
            VALID_Y[i] = 1.0
        else:
            egwrshat

In [26]:
def id_extract(filenames):
    
    indx_out = []
    indy_out = []
    lead_out = []
    day_out = []
    
    for i, name in enumerate(filenames):        
        nums = re.findall(r'\d+', name)
        lead = int(nums[-1])
        indy = int(nums[-2])
        indx = int(nums[-3])
        day = int(nums[-4])
                
        indx_out.append(indx)
        indy_out.append(indy)
        lead_out.append(lead)
        day_out.append(day)
        
    return np.array(indx_out), np.array(indy_out), np.array(lead_out), np.array(day_out)

In [27]:
indx, indy, _, day = id_extract(filename_valid_pick)

In [ ]:


valid_lead1 = np.load('{}TEST_pp15_pred_lead{}_{}.npy'.format(filepath_vec, lead1, model_tag), allow_pickle=True)[()]
valid_lead2 = np.load('{}TEST_pp15_pred_lead{}_{}.npy'.format(filepath_vec, lead2, model_tag), allow_pickle=True)[()]
valid_lead3 = np.load('{}TEST_pp15_pred_lead{}_{}.npy'.format(filepath_vec, lead3, model_tag), allow_pickle=True)[()]
valid_lead4 = np.load('{}TEST_pp15_pred_lead{}_{}.npy'.format(filepath_vec, lead4, model_tag), allow_pickle=True)[()]

VALID_lead1 = valid_lead1['y_vector']
VALID_lead2 = valid_lead2['y_vector']
VALID_lead3 = valid_lead3['y_vector']
VALID_lead4 = valid_lead4['y_vector']

VALID_lead1_y = valid_lead1['y_true']
VALID_lead2_y = valid_lead2['y_true']
VALID_lead3_y = valid_lead3['y_true']
VALID_lead4_y = valid_lead4['y_true']

IND_VALID_lead = np.load('/glade/work/ksha/NCAR/IND_VALID_lead_full.npy', allow_pickle=True)[()]

VALID_ind1 = IND_VALID_lead['lead{}'.format(lead1)]
VALID_ind2 = IND_VALID_lead['lead{}'.format(lead2)]
VALID_ind3 = IND_VALID_lead['lead{}'.format(lead3)]
VALID_ind4 = IND_VALID_lead['lead{}'.format(lead4)]

L = len(VALID_ind3)

filename_valid1_pick = []
filename_valid2_pick = []
filename_valid3_pick = []
filename_valid4_pick = []

VALID_X = np.empty((L, 512))
VALID_Y = np.zeros(L)

for i in range(L):
    
    ind_lead1 = int(VALID_ind1[i])
    ind_lead2 = int(VALID_ind2[i])
    ind_lead3 = int(VALID_ind3[i])
    ind_lead4 = int(VALID_ind4[i])
    
    filename_valid1_pick.append(filename_valid_lead1[ind_lead1])
    filename_valid2_pick.append(filename_valid_lead2[ind_lead2])
    filename_valid3_pick.append(filename_valid_lead3[ind_lead3])
    filename_valid4_pick.append(filename_valid_lead4[ind_lead4])
    
    VALID_X[i, 0:128]   = VALID_lead1[ind_lead1, :]
    VALID_X[i, 128:256] = VALID_lead2[ind_lead2, :]
    VALID_X[i, 256:384] = VALID_lead3[ind_lead3, :]
    VALID_X[i, 384:512] = VALID_lead4[ind_lead4, :]
    
    if 'pos' in filename_valid_lead3[ind_lead3]:
        if VALID_lead3_y[ind_lead3] == 1.0:
            VALID_Y[i] = 1.0
        else:
            egwrshat